# Qdislib Notebook
## Circuit Cutting Algorithm

Import the PyCOMPSs library

In [19]:
import pycompss.interactive as ipycompss

Initialize COMPSs runtime. Parameters indicates if the execution will generate task graph, tracefile, monitor interval and debug information.

In [20]:
import os

if "BINDER_SERVICE_HOST" in os.environ:
    ipycompss.start(
        graph=True,
        project_xml="../xml/project.xml",
        resources_xml="../xml/resources.xml",
    )
else:
    ipycompss.start(graph=True, monitor=1000)  # debug=True, trace=True

The runtime is already running


Import task and compss_wait_on module before annotating functions or methods

In [21]:
from pycompss.api.task import task
from pycompss.api.api import compss_wait_on

Import **qibo** module in order to handle quantic circuits.
Import **qiboconnection** module to connect to the Quantum Computer.

In [22]:
import numpy as np
import qibo
from qibo import models, gates, hamiltonians, callbacks
from qibo.models import Circuit
from qibo.symbols import X, Y, Z, I

from qiboconnection.connection import ConnectionConfiguration
from qiboconnection.api import API

qibo.__version__

'0.1.12.dev0'

Import Qdislib where the algorithm circuit cutting is implemented

In [23]:
import Qdislib
from Qdislib import wire_cutting as wc
from Qdislib import gate_cutting as gc
from Qdislib import optimal_cut as oc

Credentials needed to connect to the Quantum Computer

In [24]:
# Insert your credentials here
configuration = ConnectionConfiguration(
    username="bsc-training",
    api_key="b85b702e-6f94-42be-a876-89a670ebd9f6",
)
connection = API(configuration=configuration)
connection.ping()

[qibo-connection] 0.12.0|INFO|2024-01-10 09:04:33]: Storing personal qibo configuration...


'OK'

In [25]:
# connection.select_device_ids(device_ids=[9])
# connection.list_devices()

Define the circuit you want to cut

In [26]:
def entire_circuit():
    nqubits = 10
    circuit = models.Circuit(nqubits)

    circuit.add(gates.H(0))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.CZ(2, 6))
    circuit.add(gates.RZ(8, np.pi / 3))

    circuit.add(gates.RY(3, np.pi / 5))
    circuit.add(gates.RX(4, np.pi / 5))
    circuit.add(gates.CZ(0, 2))
    circuit.add(gates.CZ(5, 9))

    circuit.add(gates.CZ(3, 5))
    circuit.add(gates.CZ(3, 4))
    circuit.add(gates.CZ(6, 7))
    circuit.add(gates.RY(7, np.pi / 5))
    circuit.add(gates.RZ(1, np.pi / 5))

    circuit.add(gates.CZ(1, 5))
    circuit.add(gates.RX(6, np.pi / 5))
    circuit.add(gates.CZ(7, 8))

    circuit.add(gates.H(9))
    return circuit


circuit = entire_circuit()
print(circuit.draw())

q0 : ─H─o──────o─────────────────────
q1 : ───Z──────|─────────RZ─o────────
q2 : ─────o────Z────────────|────────
q3 : ─────|─RY─────o─o──────|────────
q4 : ─────|─RX─────|─Z──────|────────
q5 : ─────|──────o─Z────────Z────────
q6 : ─────Z──────|─────o──────RX─────
q7 : ────────────|─────Z─RY──────o───
q8 : ───────RZ───|───────────────Z───
q9 : ────────────Z─────────────────H─


## Algorithms for Wire Cutting

Use the functions implemented to cut and calculate the expected value of the main circuit.

5 functions:

* **circuit_cutting:**  Implements the whole algorithm in a single function. Cuts the circuit in 2 and calculates the expected value of the reconstruction.

In [27]:
circuit = entire_circuit()
# wc.circuit_cutting("ZZZZZZZZZZ",circuit, 4)

* **split** : Splits a circuit in two subcircuits. Cuts after the gate we pass as the parameter.

In [28]:
circuit = entire_circuit()
qubit, list_subcircuits = wc.split(circuit, (2, 13), True)

q0: ─H─o──────o────────
q1: ───Z──────|────────
q2: ─────o────Z────────
q3: ─────Z──────o─RX───
q4: ────────────Z─RY─o─
q5: ───────RZ────────Z─


q0: ──────────RZ─o───
q1: ─RY───o─o────|───
q2: ─RX───|─Z────|───
q3: ────o─Z──────Z───
q4: ────Z──────────H─




* **simulation** : Performs the execution of a cirucuit to calculate the expected value. It accepts one or two circuits. With 1 circuit it calculates the expected value straight forward, with 2 it performs a reeconstruction in order to provie the expected value.

In [29]:
circuit = entire_circuit()
# wc.simulation("ZZZZ", circuit)
print("\n")

circuit = entire_circuit()
qubit, list_subcircuits = wc.split(circuit, (2, 13), True)
wc.simulation("ZZZZZZZZZZ", qubit, list_subcircuits[0], list_subcircuits[1], 90000)



q0: ─H─o──────o────────
q1: ───Z──────|────────
q2: ─────o────Z────────
q3: ─────Z──────o─RX───
q4: ────────────Z─RY─o─
q5: ───────RZ────────Z─


q0: ──────────RZ─o───
q1: ─RY───o─o────|───
q2: ─RX───|─Z────|───
q3: ────o─Z──────Z───
q4: ────Z──────────H─


Expectation value after circuit cutting and reconstruction: 0.00046869308641978475


0.00046869308641978475

* **quantum_computer_execution** : Sends the execution to the quantum computer to calculate the expected value, instead of performing a simulation. (in process)
  - Interactive
  - Enqueue (returns list job ids)

In [30]:
# circuit = entire_circuit()
# cc.quantum_computer_interactive("ZZZZ", circuit1, circuit2, connection)
# jobs_id = quantum_computer_enqueue("ZZZZ", circuit1, circuit2, connection)

* **analytical_solution** : Computes the analytical expected value for the circuit. 

In [31]:
circuit = entire_circuit()
wc.analytical_solution("ZZZZZZZZZZ", circuit)

The expectation value of Z0*Z1*Z2*Z3*Z4*Z5*Z6*Z7*Z8*Z9 in the entire circuit is  0.0


0.0

## Algorithm for Gate Cutting

split_gates
simulation
frequencies
expectation_value
reconstruction

gate_cutting


In [32]:
def entire_circuit():
    nqubits = 10
    circuit = models.Circuit(nqubits)

    circuit.add(gates.H(0))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.CZ(2, 6))
    circuit.add(gates.RZ(8, np.pi / 3))

    circuit.add(gates.RY(3, np.pi / 5))
    circuit.add(gates.RX(4, np.pi / 5))
    circuit.add(gates.CZ(0, 2))
    circuit.add(gates.CZ(5, 9))

    circuit.add(gates.CZ(3, 5))
    circuit.add(gates.CZ(3, 4))
    circuit.add(gates.CZ(6, 7))
    circuit.add(gates.RY(7, np.pi / 5))
    circuit.add(gates.RZ(1, np.pi / 5))

    circuit.add(gates.CZ(1, 5))
    circuit.add(gates.RX(6, np.pi / 5))
    circuit.add(gates.CZ(7, 8))

    circuit.add(gates.H(9))
    return circuit

In [33]:
circuit = entire_circuit()
subcircuits = gc.split_gates([3, 14], circuit, True)


 Circuit 1
q0 : ─H─o────o──────────────────
q1 : ───Z────|─────────RZ─S─────
q2 : ─────S──Z──────────────────
q3 : ─────RY─────o─o────────────
q4 : ─────RX─────|─Z────────────
q5 : ──────────o─Z─────S────────
q6 : ─────S────|─────o─RX───────
q7 : ──────────|─────Z─RY───o───
q8 : ─────RZ───|────────────Z───
q9 : ──────────Z──────────────H─

 Circuit 2
q0 : ─H─o────o─────────────────────
q1 : ───Z────|─────────RZ──SDG─────
q2 : ─────S──Z─────────────────────
q3 : ─────RY─────o─o───────────────
q4 : ─────RX─────|─Z───────────────
q5 : ──────────o─Z─────SDG─────────
q6 : ─────S────|─────o─RX──────────
q7 : ──────────|─────Z─RY──────o───
q8 : ─────RZ───|───────────────Z───
q9 : ──────────Z─────────────────H─

 Circuit 3
q0 : ─H─o─────o──────────────────
q1 : ───Z─────|─────────RZ─S─────
q2 : ─────SDG─Z──────────────────
q3 : ─────RY──────o─o────────────
q4 : ─────RX──────|─Z────────────
q5 : ───────────o─Z─────S────────
q6 : ─────SDG───|─────o─RX───────
q7 : ───────────|─────Z─RY───o───
q8

In [34]:
expectation_value = []
type_gates = type(circuit.queue[3 - 1])
for subcircuit in subcircuits:
    subcircuit.add(gates.M(*range(subcircuit.nqubits)))
    result = gc.gate_simulation(subcircuit, 90000)
    frequencies = gc.gate_frequencies(result)
    expec = gc.gate_expectation_value(frequencies, 90000)
    expectation_value.append(expec)
expectation_value = compss_wait_on(expectation_value)  # IMPORTANT
reconstruct = gc.gate_reconstruction(type_gates, [3, 14], expectation_value)
print(reconstruct)



Reconstructed expected value:  (-2.145952510287815e-07+0j)
Absolute value of reconstruction  2.145952510287815e-07
Reconstruction value:  -2.145952510287815e-07
(-2.145952510287815e-07+0j)


In [35]:
circuit = entire_circuit()
print(circuit.draw())

gc.gate_cutting([3, 14], circuit, 90000, 3, True)

q0 : ─H─o──────o─────────────────────
q1 : ───Z──────|─────────RZ─o────────
q2 : ─────o────Z────────────|────────
q3 : ─────|─RY─────o─o──────|────────
q4 : ─────|─RX─────|─Z──────|────────
q5 : ─────|──────o─Z────────Z────────
q6 : ─────Z──────|─────o──────RX─────
q7 : ────────────|─────Z─RY──────o───
q8 : ───────RZ───|───────────────Z───
q9 : ────────────Z─────────────────H─

 Circuit 1
q0 : ─H─o────o──────────────────
q1 : ───Z────|─────────RZ─S─────
q2 : ─────S──Z──────────────────
q3 : ─────RY─────o─o────────────
q4 : ─────RX─────|─Z────────────
q5 : ──────────o─Z─────S────────
q6 : ─────S────|─────o─RX───────
q7 : ──────────|─────Z─RY───o───
q8 : ─────RZ───|────────────Z───
q9 : ──────────Z──────────────H─

 Circuit 2
q0 : ─H─o────o─────────────────────
q1 : ───Z────|─────────RZ──SDG─────
q2 : ─────S──Z─────────────────────
q3 : ─────RY─────o─o───────────────
q4 : ─────RX─────|─Z───────────────
q5 : ──────────o─Z─────SDG─────────
q6 : ─────S────|─────o─RX──────────
q7 : ─────────

(-3.3289531522633376e-06+0j)

## Optimal Cut

parameters: circuit, num_qubits, num_subcircuits, max_cuts, gate_cut, wire_cut

In [36]:
circuit = entire_circuit()
oc.optimal_cut(circuit)



Minimum computational cost of gate cutting:  96
Minimum computational cost of wire cutting:  60
BEST OPTION WIRE:  [(2, 13)]


[(2, 13)]

Stop COMPSs runtime.

In [37]:
ipycompss.stop(sync=True)

********************************************************
***************** STOPPING PyCOMPSs ********************
********************************************************
Checking if any issue happened.
Synchronizing all future objects left on the user scope.
Found a list to synchronize: qubit
Found a list to synchronize: list_subcircuits
Found a list to synchronize: subcircuits
Found a list to synchronize: expectation_value
Found a future object: result
Found a future object: frequencies
Found a future object: expec
	 - Could not retrieve object: expec
********************************************************
